# IMPORT MODULES REQUIRED TO PROCESS THE DATA

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(r"../input/attacks.csv", encoding = "latin-1")
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


# CHECKING THE COLUMNS TO ASSESS THE DATA

In [3]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

# LOOKING AT AMOUNT OF NaN VALUES AMONG THE DIFFERENT COLUMNS

In [4]:
df.shape

(25723, 24)

In [5]:
df.isnull().sum().sort_values(ascending=False)

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
Date                      19421
original order            19414
Case Number               17021
dtype: int64

# REMOVAL OF COMPLETELY EMPTY ROWS

In [6]:
no_empty_rows = df.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False)
no_empty_rows

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
display(no_empty_rows.isnull().sum().sort_values(ascending=False))
no_empty_rows['Case Number'].value_counts()

Unnamed: 22               8702
Unnamed: 23               8701
Time                      5755
Species                   5239
Age                       5232
Sex                       2966
Activity                  2945
Location                  2941
Fatal (Y/N)               2940
Area                      2856
Name                      2611
Country                   2451
Injury                    2429
Investigator or Source    2418
Type                      2405
Year                      2403
href formula              2402
pdf                       2401
href                      2401
Case Number.1             2401
Case Number.2             2401
Date                      2401
original order            2394
Case Number                  1
dtype: int64

0               2400
2012.09.02.b       2
1966.12.26         2
2009.12.18         2
1990.05.10         2
                ... 
1977.02.26         1
1987.00.00.b       1
1864.10.02         1
1896.00.00.b       1
1964.06.29.a       1
Name: Case Number, Length: 6287, dtype: int64

# Removing irrelevant or duplicated data

In [8]:
relevant_data_mispelled = no_empty_rows.drop(['Case Number','Year', 'pdf','href formula','href','Case Number.1','Case Number.2','Unnamed: 22', 'Unnamed: 23','original order'], axis=1)
print('\n\nDataFrame after deleting columns with duplicate values or irrelevant data\n--------------')
relevant_data_mispelled



DataFrame after deleting columns with duplicate values or irrelevant data
--------------


,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
relevant_data = relevant_data_mispelled.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False)
relevant_data

,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF"
1,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com"
2,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com"
3,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF"
4,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234"
6298,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234"
6299,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF"
6300,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938"


# Renaming mispelled column

In [9]:
display('Species has an extra whitespace',relevant_data_mispelled.columns)
relevant_data = relevant_data_mispelled.rename(columns={"Species ": "Species"})
display('Species has the whitespace removed',relevant_data.columns)

'Species has an extra whitespace'

Index(['Date', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name',
       'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'Investigator or Source'],
      dtype='object')

'Species has the whitespace removed'

Index(['Date', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name',
       'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species',
       'Investigator or Source'],
      dtype='object')

# Dividing Date in DD/MM/YY by columns

In [10]:
relevant_data.Date.value_counts()

1957                    11
1942                     9
1956                     8
1941                     7
1958                     7
                        ..
26-Aug-2011              1
25-Jan-2006              1
13-Sep-1987              1
Reported 07-Aug-1872     1
14-Feb-1992              1
Name: Date, Length: 5433, dtype: int64

In [20]:
relevant_data['Date'] = relevant_data.Date.fillna("")
for e in relevant_data['Date']:
    print(str(e))
    separation = re.findall(r'\w+',e)
    day = re.findall(r'\d{2}',str(e))
    year = re.findall(r'\d{4}',str(e))
    daymonthyear = re.findall(r"d{2}[-\s]?[A-Z]{1}\w{2}[-\s]?\d{4}", str(e))
    #res = re.findall(r"\w{3}[-\s]?\d{4}", e)
    print(daymonthyear)
    month = re.findall(r'\w{3}',str(separation))
    #print(separation)
    print(day)
    print(year)
    #print(month)

25-Jun-2018
[]
['25', '20', '18']
['2018']
18-Jun-2018
[]
['18', '20', '18']
['2018']
09-Jun-2018
[]
['09', '20', '18']
['2018']
08-Jun-2018
[]
['08', '20', '18']
['2018']
04-Jun-2018
[]
['04', '20', '18']
['2018']
03-Jun-2018
[]
['03', '20', '18']
['2018']
03-Jun-2018
[]
['03', '20', '18']
['2018']
27-May-2018
[]
['27', '20', '18']
['2018']
26-May-2018
[]
['26', '20', '18']
['2018']
26-May-2018
[]
['26', '20', '18']
['2018']
24-May-2018
[]
['24', '20', '18']
['2018']
21-May-2018
[]
['21', '20', '18']
['2018']
13-May-2018
[]
['13', '20', '18']
['2018']
13-May-2018
[]
['13', '20', '18']
['2018']
May 2018
[]
['20', '18']
['2018']
12-May-2018
[]
['12', '20', '18']
['2018']
09-May-2018
[]
['09', '20', '18']
['2018']
Reported 30-Apr-2018
[]
['30', '20', '18']
['2018']
28-Apr-2018
[]
['28', '20', '18']
['2018']
28-Apr-2018
[]
['28', '20', '18']
['2018']
25-Apr-2018
[]
['25', '20', '18']
['2018']
25-Apr-2018
[]
['25', '20', '18']
['2018']
24-Apr-2018
[]
['24', '20', '18']
['2018']
23-Apr-2018

['22', '20', '14']
['2014']
21-Mar-2014
[]
['21', '20', '14']
['2014']
18-Mar-2014
[]
['18', '20', '14']
['2014']
18-Mar-2014
[]
['18', '20', '14']
['2014']
18-Mar-2014
[]
['18', '20', '14']
['2014']
12-Mar-2014
[]
['12', '20', '14']
['2014']
13-Mar-2014
[]
['13', '20', '14']
['2014']
02-Mar-2014
[]
['02', '20', '14']
['2014']
20-Feb-2014
[]
['20', '20', '14']
['2014']
Reported 17-Feb-2014
[]
['17', '20', '14']
['2014']
08-Feb-2014
[]
['08', '20', '14']
['2014']
07-Feb-2014
[]
['07', '20', '14']
['2014']
07-Feb-2014
[]
['07', '20', '14']
['2014']
26-Jan-2014
[]
['26', '20', '14']
['2014']
25-Jan-2014
[]
['25', '20', '14']
['2014']
04-Jan-2014
[]
['04', '20', '14']
['2014']
2014
[]
['20', '14']
['2014']
2014
[]
['20', '14']
['2014']
25-Dec-2013
[]
['25', '20', '13']
['2013']
16-Dec-2013
[]
['16', '20', '13']
['2013']
13-Dec-2013
[]
['13', '20', '13']
['2013']
11-Dec-2013
[]
['11', '20', '13']
['2013']
10-Dec-2013
[]
['10', '20', '13']
['2013']
05-Dec-2013
[]
['05', '20', '13']
['2013']


07-Sep-2010
[]
['07', '20', '10']
['2010']
Reported 06-Sep-2010
[]
['06', '20', '10']
['2010']
 04-Sep-2010
[]
['04', '20', '10']
['2010']
03-Sep-2010
[]
['03', '20', '10']
['2010']
03-Sep-2010
[]
['03', '20', '10']
['2010']
02-Sep-2010
[]
['02', '20', '10']
['2010']
29-Aug-2010
[]
['29', '20', '10']
['2010']
18-Aug-2010
[]
['18', '20', '10']
['2010']
17-Aug-2010
[]
['17', '20', '10']
['2010']
14-Aug-2010
[]
['14', '20', '10']
['2010']
10-Aug-2010
[]
['10', '20', '10']
['2010']
10-Aug-2010
[]
['10', '20', '10']
['2010']
08-Aug-2010
[]
['08', '20', '10']
['2010']
07-Aug-2010
[]
['07', '20', '10']
['2010']
07-Aug-2010
[]
['07', '20', '10']
['2010']
05-Aug-2010
[]
['05', '20', '10']
['2010']
02-Aug-2010
[]
['02', '20', '10']
['2010']
02-Aug-2010
[]
['02', '20', '10']
['2010']
25-Jul-2010
[]
['25', '20', '10']
['2010']
23-Jul-2010
[]
['23', '20', '10']
['2010']
19-Jul-2010
[]
['19', '20', '10']
['2010']
17-Jul-2010
[]
['17', '20', '10']
['2010']
17-Jul-2010
[]
['17', '20', '10']
['2010']
1

28-Jul-2006
[]
['28', '20', '06']
['2006']
25-Jul-2006
[]
['25', '20', '06']
['2006']
23-Jul-2006
[]
['23', '20', '06']
['2006']
Reported 17-Jul-2006
[]
['17', '20', '06']
['2006']
17-Jul-2006
[]
['17', '20', '06']
['2006']
13-Jul-2006
[]
['13', '20', '06']
['2006']
12-Jul-2006
[]
['12', '20', '06']
['2006']
10-Jul-2006
[]
['10', '20', '06']
['2006']
09-Jul-2006
[]
['09', '20', '06']
['2006']
09-Jul-2006.
[]
['09', '20', '06']
['2006']
08-Jul-2006
[]
['08', '20', '06']
['2006']
08-Jul-2006
[]
['08', '20', '06']
['2006']
July 2006
[]
['20', '06']
['2006']
July 2006
[]
['20', '06']
['2006']
27-Jun-2006
[]
['27', '20', '06']
['2006']
26-Jun-2006
[]
['26', '20', '06']
['2006']
24-Jun-2006
[]
['24', '20', '06']
['2006']
20-Jun-2006
[]
['20', '20', '06']
['2006']
18-Jun-2006
[]
['18', '20', '06']
['2006']
17-Jun-2006
[]
['17', '20', '06']
['2006']
15-Jun-2006
[]
['15', '20', '06']
['2006']
07-Jun-2006
[]
['07', '20', '06']
['2006']
05-Jun-2006
[]
['05', '20', '06']
['2006']
31-May-2006
[]
['

[]
['11', '20', '02']
['2002']
07-Feb-2002
[]
['07', '20', '02']
['2002']
30-Jan-2002
[]
['30', '20', '02']
['2002']
30-Jan-2002
[]
['30', '20', '02']
['2002']
04-Jan-2002
[]
['04', '20', '02']
['2002']
03-Jan-2002
[]
['03', '20', '02']
['2002']
01-Jan-2002
[]
['01', '20', '02']
['2002']
01-Jan-2002
[]
['01', '20', '02']
['2002']
21-Dec-2001
[]
['21', '20', '01']
['2001']
23-Nov-2001
[]
['23', '20', '01']
['2001']
14-Nov-2001
[]
['14', '20', '01']
['2001']
07-Nov-2001
[]
['07', '20', '01']
['2001']
07-Oct-2001
[]
['07', '20', '01']
['2001']
02-Oct-2001
[]
['02', '20', '01']
['2001']
30-Sep-2001
[]
['30', '20', '01']
['2001']
30-Sep-2001
[]
['30', '20', '01']
['2001']
24-Sep-2001
[]
['24', '20', '01']
['2001']
18-Sep-2001
[]
['18', '20', '01']
['2001']
16-Sep-2001
[]
['16', '20', '01']
['2001']
15-Sep-2001
[]
['15', '20', '01']
['2001']
15-Sep-2001
[]
['15', '20', '01']
['2001']
13-Sep-2001
[]
['13', '20', '01']
['2001']
08-Sep-2001
[]
['08', '20', '01']
['2001']
07-Sep-2001
[]
['07', '

...

[]
['30', '19', '12']
['1912']
30-Aug-1912
[]
['30', '19', '12']
['1912']
23-Jul-1912
[]
['23', '19', '12']
['1912']
Reported 06-Jul-1912
[]
['06', '19', '12']
['1912']
04-May-1912
[]
['04', '19', '12']
['1912']
18-Mar-1912
[]
['18', '19', '12']
['1912']
22-Feb-1912
[]
['22', '19', '12']
['1912']
19-Feb-1912
[]
['19', '19', '12']
['1912']
03-Feb-1912
[]
['03', '19', '12']
['1912']
26-Jan-1912
[]
['26', '19', '12']
['1912']
Reported 13-Jan-1912
[]
['13', '19', '12']
['1912']
06-Jan-1912
[]
['06', '19', '12']
['1912']
01-Jan-1912
[]
['01', '19', '12']
['1912']
1912
[]
['19', '12']
['1912']
08-Nov-1911
[]
['08', '19', '11']
['1911']
26-Oct-1911
[]
['26', '19', '11']
['1911']
25-Oct-1911
[]
['25', '19', '11']
['1911']
23-Sep-1911
[]
['23', '19', '11']
['1911']
20-Sep-1911
[]
['20', '19', '11']
['1911']
17-Sep-1911
[]
['17', '19', '11']
['1911']
Reported 31-Jul-1911
[]
['31', '19', '11']
['1911']
Reported 16-Jul-1911
[]
['16', '19', '11']
['1911']
09-May-1911
[]
['09', '19', '11']
['1911']


[]
['18', '80']
['1880']
Ca. 1880
[]
['18', '80']
['1880']
Ca. 1880
[]
['18', '80']
['1880']
1880
[]
['18', '80']
['1880']
24-Dec-1879
[]
['24', '18', '79']
['1879']
10-Nov-1879
[]
['10', '18', '79']
['1879']
 22-Sep-1879
[]
['22', '18', '79']
['1879']
Reported 30-Aug-1879
[]
['30', '18', '79']
['1879']
3-Jul-1879
[]
['18', '79']
['1879']
19-Mar-1879
[]
['19', '18', '79']
['1879']
10-Mar-1879
[]
['10', '18', '79']
['1879']
1879
[]
['18', '79']
['1879']
17-Nov-1878
[]
['17', '18', '78']
['1878']
Reported 24-Oct-1878
[]
['24', '18', '78']
['1878']
13-Oct-1878
[]
['13', '18', '78']
['1878']
Reported 14-Sep-1878
[]
['14', '18', '78']
['1878']
02-Sep-1878
[]
['02', '18', '78']
['1878']
02-Sep-1878
[]
['02', '18', '78']
['1878']
09-Aug-1878
[]
['09', '18', '78']
['1878']
08-Aug-1878
[]
['08', '18', '78']
['1878']
10-Jun-1978
[]
['10', '19', '78']
['1978']
Reported 30-March-1878
[]
['30', '18', '78']
['1878']
13-Feb-1878
[]
['13', '18', '78']
['1878']
02-Feb-1878
[]
['02', '18', '78']
['1878'

['19', '06']
['1906']
Before 1906
[]
['19', '06']
['1906']
Before 1906
[]
['19', '06']
['1906']
Before 1903
[]
['19', '03']
['1903']
Before 1903
[]
['19', '03']
['1903']
1900-1905
[]
['19', '00', '19', '05']
['1900', '1905']
1883-1889
[]
['18', '83', '18', '89']
['1883', '1889']
1845-1853
[]
['18', '45', '18', '53']
['1845', '1853']

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]




[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]

[]
[]
[]


In [ ]:
for e in relevant_data['Day']:
    print(e)



In [ ]:
for e in relevant_data['Month']:
    print(e)

In [ ]:
for e in relevant_data['Year']:
    print(e)